# Join the extractions DataFrames

In [3]:
from pathlib import Path
from tqdm import tqdm
import geopandas as gpd
import pandas as pd
import warnings

warnings.simplefilter('ignore')

required_columns = ["sample_id", "ref_id", "ewoc_code", "valid_time", "extract", "h3_l3_cell", "geometry"]
dataset_folder = Path("/vitodata/worldcereal_data/EXTRACTIONS/all_datasets/all_datasets_updated_flag/")
target_folder = Path("/vitodata/worldcereal_data/EXTRACTIONS/all_datasets/grouped_datasets")
all_datasets = list(dataset_folder.glob("*.geoparquet"))

all_datasets = sorted(all_datasets, key=lambda x: x.stem)

maximum_size_mb = 500 # 1.5 GB per dataset

dataframe_groups = []
group_memory_usage_mb = 0
group = []
group_idx = 1

for f_path in tqdm(all_datasets):
    # Make sure all the datasets are in the same CRS
    df = gpd.read_parquet(f_path, columns=required_columns)
    if df.crs is None:
        df.crs = "epsg:4326"
    else:
        df = df.to_crs(epsg=4326)
    # Only keep the relevant columns necessary for extractions
    # df = df[required_columns]
    # Remove all the datasets with land-cover and not crop-type
    df = df[(df.ewoc_code > 1100000000) & (df.ewoc_code < 1200000000)]

    group.append(df)
    group_memory_usage_mb += df.memory_usage(deep=True).sum() / 1e6
    print('After adding dataset: {} group size is: {:.1f} MB'.format(f_path.stem, group_memory_usage_mb))

    if (group_memory_usage_mb > maximum_size_mb) or (f_path == all_datasets[-1]):
        print('Writing group with size: ', group_memory_usage_mb, 'MB to disk with ', len(group), 'datasets')
        group = pd.concat(group, axis=0)
        group.to_parquet(
            target_folder / f"group_{group_idx}_updated_flag.geoparquet"
        )
        # Clear up-memory and reset variables
        del group
        group = []
        group_memory_usage_mb = 0
        group_idx += 1

  1%|          | 3/254 [00:00<00:29,  8.52it/s]

After adding dataset: 00_2019_KEN_NHI-CROP-HARVEST_POINT_100 group size is: 0.0 MB
After adding dataset: 00_2021_KEN_COPERNICUS-GEOGLAM-LR_POINT_111 group size is: 0.5 MB
After adding dataset: 01_2019_AF_One-Acre-Fund-MEL_POINT_110 group size is: 1.0 MB


  2%|▏         | 5/254 [00:00<00:27,  9.16it/s]

After adding dataset: 01_2019_KEN_RadiantEarth-01_POLY_101 group size is: 1.0 MB
After adding dataset: 01_2020_RWA_FAO-WAPOR-1_POINT_111 group size is: 1.3 MB
After adding dataset: 01_2021_KEN_COPERNICUS-GEOGLAM-SR_POINT_111 group size is: 1.8 MB


  4%|▎         | 9/254 [00:00<00:19, 12.72it/s]

...

 99%|█████████▉| 252/254 [25:47<00:14,  7.11s/it]

After adding dataset: 62_2021_FIN_LPIS_POLY_110_part1 group size is: 324.8 MB


100%|█████████▉| 253/254 [25:57<00:07,  7.94s/it]

After adding dataset: 62_2021_FIN_LPIS_POLY_110_part2 group size is: 377.6 MB
After adding dataset: 62_2021_FIN_LPIS_POLY_110_part3 group size is: 407.9 MB
Writing group with size:  407.855672 MB to disk with  13 datasets


100%|██████████| 254/254 [26:26<00:00,  6.25s/it]


In [ ]:
from tqdm import tqdm
import geopandas as gpd
import pandas as pd
from pathlib import Path

dataset_folder = Path("/vitodata/worldcereal_data/EXTRACTIONS/all_datasets/grouped_datasets/")

datasets = []
for dataset in tqdm(list(dataset_folder.iterdir())):
    print(dataset.stem)
    gdf = gpd.read_parquet(dataset)
    print('dataset read')
    gdf = gdf[gdf.extract == 1]
    datasets.append(gdf)
    del gdf

datasets = pd.concat(datasets, axis=0)
datasets.to_parquet('/data/users/Public/couchard/wc_phase2_samplings.parquet')